<a href="https://colab.research.google.com/github/mikeroguez/experimento/blob/main/Accesos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*1. Normalizar e imputar*

In [ ]:
import pandas as pd
from scipy.stats import skew
import html
import numpy as np

# Rutas de entrada y salida
INPUT_PATH = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/accesos/accesos.csv"
OUTPUT_PATH = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/accesos/accesos_normalizado.csv"

# Cargar datos
df = pd.read_csv(INPUT_PATH)

# Parsear entidades HTML en la columna dispositivo
df["dispositivo"] = df["dispositivo"].apply(html.unescape)

# Normalizar valores en la columna dispositivo
df["dispositivo"] = df["dispositivo"].replace("Móvil", "Mobile")

# Convertir fechas a formato datetime
df["fecha_ingreso"] = pd.to_datetime(df["fecha_ingreso"], errors='coerce')
df["fecha_egreso"] = pd.to_datetime(df["fecha_egreso"], errors='coerce')

# Filtrar sesiones completas (fecha_egreso no nula y tiempo mayor que 0)
sesiones_completas = df[(df["fecha_egreso"].notna()) & (df["tiempo"] > 0)]

# Función para calcular skewness evitando errores
def safe_skew(x):
    if len(x) < 3:  # Si hay menos de 3 valores, no se puede calcular skewness de manera confiable
        return np.nan
    if x.std() == 0:  # Si todos los valores son idénticos, el skewness debe ser 0
        return 0
    return skew(x)

# Calcular estadísticas por alumno y dispositivo
stats = sesiones_completas.groupby(["email", "dispositivo"])["tiempo"].agg(
    media="mean",
    mediana="median",
    std="std",
    skewness=safe_skew,  # Se usa la versión corregida de skew
    cv=lambda x: x.std() / x.mean() if x.mean() > 0 else 0
).reset_index()

# Definir el método de imputación basado en skewness y CV
def elegir_metodo(row):
    if row["cv"] < 0.5 and (pd.isna(row["skewness"]) or abs(row["skewness"]) < 1):
        return "media"
    else:
        return "mediana"

stats["imputation_type"] = stats.apply(elegir_metodo, axis=1)

# Unir las estadísticas al dataset original
df = df.merge(stats[["email", "dispositivo", "imputation_type", "media", "mediana"]], on=["email", "dispositivo"], how="left")

# Crear columna is_imputed e inicializar en 0
df["is_imputed"] = 0

# Función para imputar los valores faltantes directamente en "tiempo"
def imputar_tiempo(row):
    if pd.isna(row["fecha_egreso"]) and row["tiempo"] == 0:
        row["is_imputed"] = 1  # Marcar como imputado
        if row["imputation_type"] == "media":
            return row["media"]
        elif row["imputation_type"] == "mediana":
            return row["mediana"]
    return row["tiempo"]

# Aplicar imputación directamente en "tiempo"
df["tiempo"] = df.apply(imputar_tiempo, axis=1)

# Corregir la columna is_imputed después de la imputación
df["is_imputed"] = ((df["tiempo"] > 0) & (pd.isna(df["fecha_egreso"]))).astype(int)

# Actualizar la columna fecha_egreso para los datos imputados
df["fecha_egreso"] = df.apply(
    lambda row: row["fecha_ingreso"] + pd.to_timedelta(row["tiempo"], unit='m')
    if row["is_imputed"] == 1 else row["fecha_egreso"],
    axis=1
)

# Guardar el archivo procesado
df.to_csv(OUTPUT_PATH, index=False)

print(f"Archivo procesado y guardado en {OUTPUT_PATH}")


Archivo procesado y guardado en /content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/accesos/accesos_normalizado.csv


*2. Consolidar*

In [ ]:
import pandas as pd

# Definir rutas de entrada y salida
INPUT_PATH = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/accesos/accesos_normalizado.csv"
OUTPUT_PATH = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/accesos/accesos_consolidado.csv"

# Cargar el dataset
df = pd.read_csv(INPUT_PATH)

# Asegurar que la columna dispositivo esté bien escrita
df["dispositivo"] = df["dispositivo"].str.strip()

# Crear nuevas columnas para identificar sesiones desktop y móvil
df["desktop_sessions"] = (df["dispositivo"] == "Desktop").astype(int)
df["mobile_sessions"] = (df["dispositivo"] == "Mobile").astype(int)

# Crear nuevas columnas para contar sesiones imputadas por tipo de dispositivo
df["desktop_imputed_sessions"] = ((df["dispositivo"] == "Desktop") & (df["is_imputed"] == 1)).astype(int)
df["mobile_imputed_sessions"] = ((df["dispositivo"] == "Mobile") & (df["is_imputed"] == 1)).astype(int)

# Consolidar los datos por curso y alumno
df_consolidado = df.groupby(["curso", "email", "groupKey"], as_index=False).agg({
    "tiempo": ["sum", "mean"],  # total_time, average_time
    "fecha_ingreso": "nunique",  # active_days
    "desktop_sessions": "sum",
    "mobile_sessions": "sum",
    "desktop_imputed_sessions": "sum",
    "mobile_imputed_sessions": "sum"
})

# Renombrar columnas
df_consolidado.columns = [
    "curso", "email", "groupKey",
    "total_time", "average_time", "active_days_count",
    "desktop_sessions_count", "mobile_sessions_count",
    "desktop_imputed_session_time_count", "mobile_imputed_session_time_count"
]

# Calcular total de sesiones
df_consolidado["total_sessions_count"] = df_consolidado["desktop_sessions_count"] + df_consolidado["mobile_sessions_count"]

# Calcular total de sesiones imputadas
df_consolidado["total_imputed_session_time_count"] = df_consolidado["desktop_imputed_session_time_count"] + df_consolidado["mobile_imputed_session_time_count"]

# Calcular la proporción de sesiones imputadas
df_consolidado["imputed_ratio"] = df_consolidado["total_imputed_session_time_count"] / df_consolidado["total_sessions_count"]
df_consolidado["imputed_ratio"] = df_consolidado["imputed_ratio"].fillna(0)  # Reemplazar valores NaN con 0

# Marcar casos extremos: 100% imputación o ≤ 2 sesiones totales
df_consolidado["extreme_case"] = ((df_consolidado["imputed_ratio"] == 1.0) | (df_consolidado["total_sessions_count"] <= 2)).astype(int)

# Asegurar que la columna extreme_case no tenga valores vacíos
df_consolidado["extreme_case"] = df_consolidado["extreme_case"].fillna(0)

# Guardar el dataset consolidado
df_consolidado.to_csv(OUTPUT_PATH, index=False)

print("Consolidación completada y guardada en:", OUTPUT_PATH)


Consolidación completada y guardada en: /content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/accesos/accesos_consolidado.csv


3. Lipieza

In [ ]:
import pandas as pd

# Definir rutas de entrada y salida
INPUT_PATH = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/accesos/accesos_consolidado.csv"
OUTPUT_PATH = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/accesos/accesos_consolidado_cleaned.csv"

# Cargar el dataset consolidado
df = pd.read_csv(INPUT_PATH)

# Filtrar registros donde extreme_case != 1
df_cleaned = df[df["extreme_case"] != 1].drop(columns=["extreme_case"])

# Guardar el dataset limpio
df_cleaned.to_csv(OUTPUT_PATH, index=False)

print("Dataset limpio guardado en:", OUTPUT_PATH)


Dataset limpio guardado en: /content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/accesos/accesos_consolidado_cleaned.csv
